In [1]:
!pip install -q transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "sdornala/bert-uncertainty-classifier_teacher"

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/847 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [5]:
def classify_uncertainty(text):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

    # Get predictions
    outputs = model(**inputs)
    probabilities = outputs.logits.softmax(dim=1)

    # Convert to labels
    predicted_class = probabilities.argmax().item()
    confidence = probabilities.max().item()

    return {
        "text": text,
        "prediction": model.config.id2label[predicted_class],
        "confidence": f"{confidence:.3f}"
    }

# Test example
sample_text = "This might possibly be an example of hedging language."
print(classify_uncertainty(sample_text))


{'text': 'This might possibly be an example of hedging language.', 'prediction': 'Uncertain', 'confidence': '0.790'}


In [6]:
def classify_uncertainty(text):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

    # Get predictions
    outputs = model(**inputs)
    probabilities = outputs.logits.softmax(dim=1)

    # Get raw class prediction (0 or 1)
    predicted_class = probabilities.argmax().item()
    confidence = probabilities.max().item()

    return predicted_class  # Returns 0 (Uncertain) or 1 (Certain)

# If you want both numeric label and confidence:
def classify_with_confidence(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    probabilities = outputs.logits.softmax(dim=1)
    return {
        "prediction": probabilities.argmax().item(),
        "confidence": probabilities.max().item()
    }

# Example usage:
sample_text = "This might possibly be an example of hedging language."
print(classify_uncertainty(sample_text))  # Output: 0
print(classify_with_confidence(sample_text))  # Output: {'prediction': 0, 'confidence': 0.790}


0
{'prediction': 0, 'confidence': 0.7902387976646423}
